In [2]:
from numbers import Number
from typing import Optional, List
from .autograd import NDArray
from .autograd import Op, Tensor, Value, TensorOp
from .autograd import TensorTuple, TensorTupleOp
import numpy

# NOTE: we will import numpy as the array_api
# as the backend for our computations, this line will change in later homeworks
import numpy as array_api

ImportError: attempted relative import with no known parent package

## AutoGrad

In [110]:
from typing import List, Optional, NamedTuple, Tuple, Union
from collections import namedtuple
import numpy

# needle version
LAZY_MODE = False
TENSOR_COUNTER = 0

# NOTE: we will import numpy as the array_api
# as the backend for our computations, this line will change in later homeworks
import numpy as array_api
NDArray = numpy.ndarray


class Device:
    """Indicates the device supporting an NDArray."""


class CPUDevice(Device):
    """Represents data that sits in CPU"""

    def __repr__(self):
        return "needle.cpu()"

    def __hash__(self):
        return self.__repr__().__hash__()

    def __eq__(self, other):
        return isinstance(other, CPUDevice)

    def enabled(self):
        return True

def cpu():
    """Return cpu device"""
    return CPUDevice()

def all_devices():
    """return a list of all available devices"""
    return [cpu()]


class Op:
    """Operator definition."""

    def __call__(self, *args):
        raise NotImplementedError()

    def compute(self, *args: Tuple[NDArray]):
        """Calculate forward pass of operator.

        Parameters
        ----------
        input: np.ndarray
            A list of input arrays to the function

        Returns
        -------
        output: nd.array
            Array output of the operation

        """
        raise NotImplementedError()

    def gradient(
        self, out_grad: "Value", node: "Value"
    ) -> Union["Value", Tuple["Value"]]:
        """Compute partial adjoint for each input value for a given output adjoint.

        Parameters
        ----------
        out_grad: Value
            The adjoint wrt to the output value.

        node: Value
            The value node of forward evaluation.

        Returns
        -------
        input_grads: Value or Tuple[Value]
            A list containing partial gradient adjoints to be propagated to
            each of the input node.
        """
        raise NotImplementedError()

    def gradient_as_tuple(self, out_grad: "Value", node: "Value") -> Tuple["Value"]:
        """ Convenience method to always return a tuple from gradient call"""
        output = self.gradient(out_grad, node)
        if isinstance(output, tuple):
            return output
        elif isinstance(output, list):
            return tuple(output)
        else:
            return (output,)


class TensorOp(Op):
    """ Op class specialized to output tensors, will be alternate subclasses for other structures """

    def __call__(self, *args):
        return Tensor.make_from_op(self, args)


class TensorTupleOp(Op):
    """Op class specialized to output TensorTuple"""

    def __call__(self, *args):
        return TensorTuple.make_from_op(self, args)


class Value:
    """A value in the computational graph."""

    # trace of computational graph
    op: Optional[Op]
    inputs: List["Value"]
    # The following fields are cached fields for
    # dynamic computation
    cached_data: NDArray
    requires_grad: bool

    def realize_cached_data(self):
        """Run compute to realize the cached data"""
        # avoid recomputation
        if self.cached_data is not None:
            return self.cached_data
        # note: data implicitly calls realized cached data
        self.cached_data = self.op.compute(
            *[x.realize_cached_data() for x in self.inputs]
        )
        return self.cached_data

    def is_leaf(self):
        return self.op is None

    def __del__(self):
        global TENSOR_COUNTER
        TENSOR_COUNTER -= 1

    def _init(
        self,
        op: Optional[Op],
        inputs: List["Tensor"],
        *,
        num_outputs: int = 1,
        cached_data: List[object] = None,
        requires_grad: Optional[bool] = None
    ):
        global TENSOR_COUNTER
        TENSOR_COUNTER += 1
        if requires_grad is None:
            requires_grad = any(x.requires_grad for x in inputs)
        self.op = op
        self.inputs = inputs
        self.num_outputs = num_outputs
        self.cached_data = cached_data
        self.requires_grad = requires_grad

    @classmethod
    def make_const(cls, data, *, requires_grad=False):
        value = cls.__new__(cls)
        value._init(
            None,
            [],
            cached_data=data,
            requires_grad=requires_grad,
        )
        return value

    @classmethod
    def make_from_op(cls, op: Op, inputs: List["Value"]):
        value = cls.__new__(cls)
        value._init(op, inputs)

        if not LAZY_MODE:
            if not value.requires_grad:
                return value.detach()
            value.realize_cached_data()
        return value


### Not needed in HW1
class TensorTuple(Value):
    """Represent a tuple of tensors.

    To keep things simple, we do not support nested tuples.
    """

    def __len__(self):
        cdata = self.realize_cached_data()
        return len(cdata)

    def __getitem__(self, index: int):
        return needle.ops.tuple_get_item(self, index)

    def tuple(self):
        return tuple([x for x in self])

    def __repr__(self):
        return "needle.TensorTuple" + str(self.tuple())

    def __str__(self):
        return self.__repr__()

    def __add__(self, other):
        assert isinstance(other, TensorTuple)
        assert len(self) == len(other)
        return needle.ops.make_tuple(*[self[i] + other[i] for i in range(len(self))])

    def detach(self):
        """Create a new tensor that shares the data but detaches from the graph."""
        return Tuple.make_const(self.realize_cached_data())


class Tensor(Value):
    grad: "Tensor"

    def __init__(
        self,
        array,
        *,
        device: Optional[Device] = None,
        dtype=None,
        requires_grad=True,
        **kwargs
    ):
        if isinstance(array, Tensor):
            if device is None:
                device = array.device
            if dtype is None:
                dtype = array.dtype
            if device == array.device and dtype == array.dtype:
                cached_data = array.realize_cached_data()
            else:
                # fall back, copy through numpy conversion
                cached_data = Tensor._array_from_numpy(
                    array.numpy(), device=device, dtype=dtype
                )
        else:
            device = device if device else cpu()
            cached_data = Tensor._array_from_numpy(array, device=device, dtype=dtype)

        self._init(
            None,
            [],
            cached_data=cached_data,
            requires_grad=requires_grad,
        )

    @staticmethod
    def _array_from_numpy(numpy_array, device, dtype):
        if array_api is numpy:
            return numpy.array(numpy_array, dtype=dtype)
        return array_api.array(numpy_array, device=device, dtype=dtype)

    @staticmethod
    def make_from_op(op: Op, inputs: List["Value"]):
        tensor = Tensor.__new__(Tensor)
        tensor._init(op, inputs)
        if not LAZY_MODE:
            tensor.realize_cached_data()
        return tensor

    @staticmethod
    def make_const(data, requires_grad=False):
        tensor = Tensor.__new__(Tensor)
        tensor._init(
            None,
            [],
            cached_data=data
            if not isinstance(data, Tensor)
            else data.realize_cached_data(),
            requires_grad=requires_grad,
        )
        return tensor

    @property
    def data(self):
        return self.detach()

    @data.setter
    def data(self, value):
        assert isinstance(value, Tensor)
        assert value.dtype == self.dtype, "%s %s" % (
            value.dtype,
            self.dtype,
        )
        self.cached_data = value.realize_cached_data()

    def detach(self):
        """Create a new tensor that shares the data but detaches from the graph."""
        return Tensor.make_const(self.realize_cached_data())

    @property
    def shape(self):
        return self.realize_cached_data().shape

    @property
    def dtype(self):
        return self.realize_cached_data().dtype

    @property
    def device(self):
        data = self.realize_cached_data()
        # numpy array always sits on cpu
        if array_api is numpy:
            return cpu()
        return data.device

    def backward(self, out_grad=None):
        out_grad = out_grad if out_grad else Tensor(numpy.ones(self.shape))
        compute_gradient_of_variables(self, out_grad)

    def __repr__(self):
        return "needle.Tensor(" + str(self.realize_cached_data()) + ")"

    def __str__(self):
        return self.realize_cached_data().__str__()

    def numpy(self):
        data = self.realize_cached_data()
        if array_api is numpy:
            return data
        return data.numpy()

    def __add__(self, other):
        if isinstance(other, Tensor):
            return needle.ops.EWiseAdd()(self, other)
        else:
            return needle.ops.AddScalar(other)(self)

    def __mul__(self, other):
        if isinstance(other, Tensor):
            return needle.ops.EWiseMul()(self, other)
        else:
            return needle.ops.MulScalar(other)(self)

    def __pow__(self, other):
        if isinstance(other, Tensor):
            raise NotImplementedError()
        else:
            return needle.ops.PowerScalar(other)(self)

    def __sub__(self, other):
        if isinstance(other, Tensor):
            return needle.ops.EWiseAdd()(self, needle.ops.Negate()(other))
        else:
            return needle.ops.AddScalar(-other)(self)

    def __truediv__(self, other):
        if isinstance(other, Tensor):
            return needle.ops.EWiseDiv()(self, other)
        else:
            return needle.ops.DivScalar(other)(self)

    def __matmul__(self, other):
        return needle.ops.MatMul()(self, other)

    def matmul(self, other):
        return needle.ops.MatMul()(self, other)

    def sum(self, axes=None):
        return needle.ops.Summation(axes)(self)

    def broadcast_to(self, shape):
        return needle.ops.BroadcastTo(shape)(self)

    def reshape(self, shape):
        return needle.ops.Reshape(shape)(self)

    def __neg__(self):
        return needle.ops.Negate()(self)

    def transpose(self, axes=None):
        return needle.ops.Transpose(axes)(self)

    __radd__ = __add__
    __rmul__ = __mul__
    __rsub__ = __sub__
    __rmatmul__ = __matmul__


def compute_gradient_of_variables(output_tensor, out_grad):
    """Take gradient of output node with respect to each node in node_list.

    Store the computed result in the grad field of each Variable.
    """
    # a map from node to a list of gradient contributions from each output node
    node_to_output_grads_list: Dict[Tensor, List[Tensor]] = {}
    # Special note on initializing gradient of
    # We are really taking a derivative of the scalar reduce_sum(output_node)
    # instead of the vector output_node. But this is the common case for loss function.
    node_to_output_grads_list[output_tensor] = [out_grad]

    # Traverse graph in reverse topological order given the output_node that we are taking gradient wrt.
    reverse_topo_order = list(reversed(find_topo_sort([output_tensor])))

    ### BEGIN YOUR SOLUTION
    
    for node in reverse_topo_order:
        node.gradient(out_grad, output_tensor)
    ### END YOUR SOLUTION


def find_topo_sort(node_list: List[Value]) -> List[Value]:
    """Given a list of nodes, return a topological sort list of nodes ending in them.

    A simple algorithm is to do a post-order DFS traversal on the given nodes,
    going backwards based on input edges. Since a node is added to the ordering
    after all its predecessors are traversed due to post-order DFS, we get a topological
    sort.
    """
    topo = []
    visited = set()

    for node in node.inputs:
        if node not in visited: topo_sort_dfs(node, visited, topo)
    
    topo_order.reverse()  # Reverse the list to get the correct topological order
    return topo_order


def topo_sort_dfs(node, visited, topo_order):
    """Post-order DFS"""
    ### BEGIN YOUR SOLUTION
    
    visited.add(node)
    for child in node.inputs:
        if child not in visited:
            topo_sort_dfs(child, visited, topo_order)
    topo_order.append(node)
    
    ### END YOUR SOLUTION


##############################
####### Helper Methods #######
##############################


def sum_node_list(node_list):
    """Custom sum function in order to avoid create redundant nodes in Python sum implementation."""
    from operator import add
    from functools import reduce

    return reduce(add, node_list)


In [112]:
# Test case 1
a1, b1 = ndl.Tensor(np.asarray([[0.88282157]])), ndl.Tensor(np.asarray([[0.90170084]]))
c1 = 3*a1*a1 + 4*b1*a1 - a1

soln = np.array([np.array([[0.88282157]]),
                 np.array([[2.64846471]]),
                 np.array([[2.33812177]]),
                 np.array([[0.90170084]]),
                 np.array([[3.60680336]]),
                 np.array([[3.1841638]]),
                 np.array([[5.52228558]]),
                 np.array([[-0.88282157]]),
                 np.array([[4.63946401]])])

topo_order = np.array([x.numpy() for x in ndl.autograd.find_topo_sort([c1])])

NameError: name 'ndl' is not defined

In [ ]:
assert len(soln) == len(topo_order)
np.testing.assert_allclose(topo_order, soln, rtol=1e-06, atol=1e-06)

## Element Wise Addition

Let's walk through the step-by-step derivative calculation for the `EWiseAdd` operation:

We have the function `f(a, b) = a + b`, where `a` and `b` are tensors. Our goal is to compute the partial derivatives with respect to `a` and `b`.

Let's start by calculating the derivative of `f` with respect to `a`, denoted as `df/da`:

Step 1: Compute the derivative of `f` with respect to `a`.

$\frac{{\partial f}}{{\partial a}} = \frac{{\partial}}{{\partial a}} (a + b)$

Since `a` is the variable we are differentiating with respect to, the derivative of `a` with respect to itself is 1:

$$\frac{{\partial f}}{{\partial a}} = 1$$

Therefore, $$\frac{{\partial f}}{{\partial a}} = 1.$$

Step 2: Compute the derivative of `f` with respect to `b`.

$$\frac{{\partial f}}{{\partial b}} = \frac{{\partial}}{{\partial b}} (a + b)$$

Again, since `b` is the variable we are differentiating with respect to, the derivative of `b` with respect to itself is 1:

$$\frac{{\partial f}}{{\partial b}} = 1$$

Therefore, $$\frac{{\partial f}}{{\partial b}} = 1$$

Hence, the partial derivatives of `f(a, b) = a + b` with respect to `a` and `b` are both equal to 1.

In [14]:
class EWiseAdd(TensorOp):
    def compute(self, a: NDArray, b: NDArray):
        return a + b

    def gradient(self, out_grad: Tensor, node: Tensor):
        return out_grad, out_grad


def add(a, b):
    return EWiseAdd()(a, b)

## Scalar Addition

Certainly! Here's the proof and explanation for the derivative of the `AddScalar` operator:

Let's denote the scalar as `c` and `a` as the tensor being added by the scalar. The operation can be described as `f(a) = a + c`.

The function for the backward pass (i.e., the gradient) is `df/da = 1`, which means the derivative of `f(a)` with respect to `a` is simply `1`.

The LaTeX document will look as follows:

We are given a function $f(a) = a + c$, where $a$ is a tensor and $c$ is a scalar. Our task is to find the derivative of this function with respect to $a$.

By differentiating the function $f(a)$ with respect to $a$, we find:

\begin{align*}
\frac{df}{da} &= \frac{d}{da} (a + c) \\
&= 1
\end{align*}

Therefore, the gradient of $f(a)$ with respect to $a$ is $1$.


We starts by defining the function `f(a) = a + c`. It then explains that when we differentiate `f(a)` with respect to `a`, we find that the derivative is `1`. This means that the gradient of `f(a)` with respect to `a` is `1`, which matches the behavior of the `AddScalar` operator as provided in the `gradient` method.

In [15]:
class AddScalar(TensorOp):
    def __init__(self, scalar):
        self.scalar = scalar

    def compute(self, a: NDArray):
        return a + self.scalar

    def gradient(self, out_grad: Tensor, node: Tensor):
        return out_grad


def add_scalar(a, scalar):
    return AddScalar(scalar)(a)

## Element Wise Mult

Certainly! Here's the proof and explanation for the derivative of the `EWiseMul` (element-wise multiplication) operator:

Let's denote the two input tensors as `a` and `b`. The operation can be described as `f(a, b) = a * b`, where `*` represents element-wise multiplication.

The function for the backward pass (i.e., the gradient) is `df/da = b` and `df/db = a`. This means that the derivative of `f(a, b)` with respect to `a` is `b`, and the derivative with respect to `b` is `a`.

The LaTeX document will look as follows:


We are given a function $f(a, b) = a \odot b$, where $a$ and $b$ are tensors, and $\odot$ represents element-wise multiplication. Our task is to find the derivatives of this function with respect to $a$ and $b$.

By differentiating the function $f(a, b)$ with respect to $a$, we find:

\begin{align*}
\frac{df}{da} &= \frac{d}{da} (a \odot b) \\
&= b
\end{align*}

Therefore, the gradient of $f(a, b)$ with respect to $a$ is $b$.

Similarly, by differentiating the function $f(a, b)$ with respect to $b$, we find:

\begin{align*}
\frac{df}{db} &= \frac{d}{db} (a \odot b) \\
&= a
\end{align*}

Therefore, the gradient of $f(a, b)$ with respect to $b$ is $a$.

In [16]:
class EWiseMul(TensorOp):
    def compute(self, a: NDArray, b: NDArray):
        return a * b

    def gradient(self, out_grad: Tensor, node: Tensor):
        lhs, rhs = node.inputs
        return out_grad * rhs, out_grad * lhs


def multiply(a, b):
    return EWiseMul()(a, b)

## Scalar Mult

Certainly! Here's the proof and explanation for the derivative of the `MulScalar` operator:

Let's denote the scalar as `c` and `a` as the tensor being multiplied by the scalar. The operation can be described as `f(a) = a * c`.

The function for the backward pass (i.e., the gradient) is `df/da = c`, which means the derivative of `f(a)` with respect to `a` is `c`.

The LaTeX document will look as follows:

We are given a function $f(a) = a \cdot c$, where $a$ is a tensor and $c$ is a scalar. Our task is to find the derivative of this function with respect to $a$.

By differentiating the function $f(a)$ with respect to $a$, we find:

\begin{align*}
\frac{df}{da} &= \frac{d}{da} (a \cdot c) \\
&= c
\end{align*}

Therefore, the gradient of $f(a)$ with respect to $a$ is $c$.

We starts by defining the function `f(a) = a * c`. It then explains that when we differentiate `f(a)` with respect to `a`, we find that the derivative is `c`. This means that the gradient of `f(a)` with respect to `a` is `c`, which matches the behavior of the `MulScalar` operator as provided in the `gradient` method.

In [17]:
class MulScalar(TensorOp):
    def __init__(self, scalar):
        self.scalar = scalar

    def compute(self, a: NDArray):
        return a * self.scalar

    def gradient(self, out_grad: Tensor, node: Tensor):
        return (out_grad * self.scalar,)


def mul_scalar(a, scalar):
    return MulScalar(scalar)(a)

## Power Scalar

Certainly! Here's the proof and explanation for the derivative of the `PowerScalar` operator:

Let's denote the scalar as `n` and `a` as the tensor being raised to the power of the scalar. The operation can be described as `f(a) = a^n`.

The function for the backward pass (i.e., the gradient) is `df/da = n * a^(n-1)`.

The LaTeX document will look as follows:


We are given a function $f(a) = a^n$, where $a$ is a tensor and $n$ is a scalar. Our task is to find the derivative of this function with respect to $a$.

By differentiating the function $f(a)$ with respect to $a$, we find:

\begin{align*}
\frac{df}{da} &= \frac{d}{da} (a^n) \\
&= n \cdot a^{n-1}
\end{align*}

Therefore, the gradient of $f(a)$ with respect to $a$ is $n \cdot a^{n-1}$.

We starts by defining the function `f(a) = a^n`, where `^` represents exponentiation. It then explains that when we differentiate `f(a)` with respect to `a`, we find that the derivative is `n * a^(n-1)`. This means that the gradient of `f(a)` with respect to `a` is `n * a^(n-1)`, which matches the behavior of the `PowerScalar` operator as provided in the `gradient` method.

In [42]:
class PowerScalar(TensorOp):
    """Op raise a tensor to an (integer) power."""

    def __init__(self, scalar: int):
        self.scalar = scalar

    def compute(self, a: NDArray) -> NDArray:
        ### BEGIN YOUR SOLUTION
        return array_api.power(a, self.scalar)
        ### END YOUR SOLUTION

    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        return (self.scalar * power_scalar(node, self.scalar - 1) * out_grad, )
        ### END YOUR SOLUTION


def power_scalar(a, scalar):
    return PowerScalar(scalar)(a)

## Element Wise Divide

The operation described here is an element-wise division of two tensors, `a` and `b`, where the operation can be described as `f(a, b) = a / b`. 

We'll compute the partial derivatives with respect to `a` and `b`:

1. The partial derivative of `f(a, b)` with respect to `a` (`df/da`) is `1/b`.

2. The partial derivative of `f(a, b)` with respect to `b` (`df/db`) is `-a / b^2`.

These results align with the backward function you've provided.

Now, let's translate this into LaTeX:


We are given a function $f(a, b) = \frac{a}{b}$, where $a$ and $b$ are tensors. Our task is to find the partial derivatives of this function with respect to $a$ and $b$.

Let's start with $\frac{\partial f}{\partial a}$:

\begin{align*}
\frac{\partial f}{\partial a} &= \frac{\partial}{\partial a} \left(\frac{a}{b}\right) \\
&= \frac{1}{b}
\end{align*}

Now, let's compute $\frac{\partial f}{\partial b}$:

\begin{align*}
\frac{\partial f}{\partial b} &= \frac{\partial}{\partial b} \left(\frac{a}{b}\right) \\
&= - \frac{a}{b^{2}}
\end{align*}

Here is a detailed derivative:

Given a function of the form $y = \frac{u}{v}$, where both $u$ and $v$ are functions of $x$, the quotient rule of differentiation states:

$$\frac{dy}{dx} = \frac{v \cdot \frac{du}{dx} - u \cdot \frac{dv}{dx}}{v^2}$$

In our case, we're looking at the function $y = \frac{a}{b}$, where $a$ and $b$ are tensors. We want to find the derivative with respect to $b$ (instead of $x$ in our general formula). So we have:

$$\frac{dy}{db} = \frac{b \cdot \frac{da}{db} - a \cdot \frac{db}{db}}{b^2}$$

Since $a$ does not depend on $b$, $\frac{da}{db} = 0$, and since any variable is equal to itself, $\frac{db}{db} = 1$. 

So the derivative $\frac{dy}{db}$ simplifies to:

$$\frac{dy}{db} = \frac{b \cdot 0 - a \cdot 1}{b^2}$$

Therefore, the derivative of $y$ with respect to $b$ is $-\frac{a}{b^2}$.

Therefore, the gradient of $f(a, b)$ with respect to $a$ is $\frac{1}{b}$, and the gradient of $f(a, b)$ with respect to $b$ is $- \frac{a}{b^{2}}$.

In [105]:
class EWiseDiv(TensorOp):
    """Op to element-wise divide two nodes."""

    def compute(self, a, b):
        ### BEGIN YOUR SOLUTION
        return a / b
        ### END YOUR SOLUTION

    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        a, b = node.inputs
        return divide(out_grad, b), out_grad * negate(divide(a, power_scalar(b, 2)))
        ### END YOUR SOLUTION


def divide(a, b):
    return EWiseDiv()(a, b)

## Divide Scalar

Let's denote the scalar as `c`, and `a` as the tensor being divided by the scalar. The operation can be described as `f(a) = a / c`.

The function for the backward pass (i.e., the gradient) is `df/da = 1/c`.

This is the derivative of `f(a)` with respect to `a`.

The LaTeX document will look as follows:

We are given a function $f(a) = \frac{a}{c}$, where $a$ is a tensor and $c$ is a scalar. Our task is to find the derivative of this function with respect to $a$.

By using the power rule of differentiation, where the derivative of $a^n$ is $n \cdot a^{n-1}$, we can rewrite $f(a)$ as $f(a) = c^{-1}a$. 

Now, we can differentiate this with respect to $a$:

\begin{align*}
\frac{df}{da} &= \frac{d}{da} (c^{-1}a) \\
&= c^{-1} \frac{d}{da} (a) \\
&= c^{-1} \\
&= \frac{1}{c}
\end{align*}

Therefore, the gradient of $f(a)$ with respect to $a$ is $\frac{1}{c}$.

In [20]:
class DivScalar(TensorOp):
    def __init__(self, scalar):
        self.scalar = scalar

    def compute(self, a):
        ### BEGIN YOUR SOLUTION
        return a / self.scalar
        ### END YOUR SOLUTION

    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        return out_grad * (1/self.scalar)
        ### END YOUR SOLUTION


def divide_scalar(a, scalar):
    return DivScalar(scalar)(a)

## Transpose

In [31]:
class Transpose(TensorOp):
    def __init__(self, axes: Optional[tuple] = None):
        self.axes = axes

    def compute(self, a):
        # Swap the last two dimensions of the input tensor
        if self.axes:
            a = a.swapaxes(self.axes[0], self.axes[1])
        else:
            a = a.swapaxes(-2, -1)
        return a


    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        return transpose(out_grad,axes=self.axes)
        ### END YOUR SOLUTION


def transpose(a, axes=None):
    return Transpose(axes)(a)

## Reshape

In [33]:
class Reshape(TensorOp):
    def __init__(self, shape):
        self.shape = shape

    def compute(self, a):
        ### BEGIN YOUR SOLUTION
        self.input_shape = a.shape
        return array_api.reshape(a, newshape=self.shape)
        ### END YOUR SOLUTION

    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        # reshape gradient to match input shape
        return reshape(out_grad, self.input_shape), 
        ### END YOUR SOLUTION


def reshape(a, shape):
    return Reshape(shape)(a)

## Broadcast

Here's the requested information with the appropriate formatting:

#### Implementing Backward Pass for Broadcasting Operation

Broadcasting is an operation that expands a tensor to a given shape by replicating its values along the new dimensions. The backward pass of broadcasting needs to "undo" this operation and sum up the gradients that have been duplicated due to the broadcasting operation.

In the `gradient` function for the broadcasting operation, `out_grad` is the gradient tensor flowing back from further down the computational graph. This tensor has the same shape as the output of the broadcasting operation, i.e., the expanded or broadcasted shape. 

We need to sum up the gradients in `out_grad` along the dimensions that have been extended by the broadcasting operation, and return a gradient tensor that has the same shape as the original input tensor.

To achieve this, the `gradient` function does the following:

1. **Identify the singleton dimensions:** Singleton dimensions are dimensions of size 1 that were either expanded due to broadcasting or added when broadcasting to a higher rank tensor. We need to identify these dimensions so we know which axes to sum over in the backward pass.

2. **Sum the gradients over singleton dimensions:** We sum the `out_grad` tensor over all the singleton dimensions. This effectively reverses the broadcasting operation, as it adds up all the gradients that were copied due to the broadcasting.

3. **Reshape the gradient tensor:** We then reshape the resulting gradient tensor to ensure that it has the same shape as the input tensor. This is important because the gradient tensor must match the shape of the input tensor for it to be propagated correctly in the backward pass. 

The `gradient` function for the broadcasting operation can be implemented as follows:

```python
def gradient(self, out_grad, node):
    # Shape of the input tensor
    in_shape = node.inputs[0].shape

    # Compute the difference in tensor ranks (dimensions) between the output and the input
    rank_diff = len(self.shape) - len(in_shape)

    # Indices of singleton dimensions added by broadcasting to higher-rank tensor
    new_singleton_dims = list(range(rank_diff))

    # Indices of singleton dimensions in the input tensor that were expanded by broadcasting
    expanded_singleton_dims = [i + rank_diff for i, size in enumerate(in_shape) if size == 1]

    # Combine all indices of singleton dimensions
    singleton_dims = new_singleton_dims + expanded_singleton_dims

    # Sum the out_grad tensor over all singleton dimensions to "undo" the broadcasting
    grad = summation(out_grad, axes=tuple(singleton_dims))

    # Reshape the resulting gradient tensor to match the shape of the input tensor
    grad = reshape(grad, in_shape)

    # Return the gradient tensor
    return (grad,)
```

Remember to validate the correctness of this implementation by performing gradient checking as suggested.

In [74]:
import torch

# Initialize a 2x2 tensor
A = torch.tensor([[1., 2.], [3., 4.]])

# Initialize a scalar
b = torch.tensor([2.])

# Perform the broadcasting operation
C = A + b
C, C.shape

(tensor([[3., 4.],
         [5., 6.]]),
 torch.Size([2, 2]))

In [78]:
# Make sure to enable gradient computation for b
b = torch.tensor([2.], requires_grad=True)

# Perform the broadcasting operation
C = A + b

# Now let's assume some gradient coming from the next layer during backpropagation
grad_next = torch.tensor([[1., 2.], [3., 4.]])

# Perform the backward pass
C.backward(grad_next)

# Check the gradient for b
b.grad

tensor([10.])

In [34]:
class BroadcastTo(TensorOp):
    def __init__(self, shape):
        self.shape = shape

    def compute(self, a):
        self.input_shape = a.shape
        return array_api.broadcast_to(a, self.shape)

    def gradient(self, out_grad, node):
        # Get the difference in ranks between the broadcasted tensor and the input tensor
        rank_diff = len(self.shape) - len(self.input_shape)

        # Identify the new singleton dimensions that were added due to broadcasting
        # to a higher rank tensor. These are dimensions that do not exist in the input tensor.
        new_singleton_dims = list(range(rank_diff))

        # Identify the singleton dimensions in the input tensor that were expanded by broadcasting.
        # We count from the end of the shape (-1 refers to the last dimension, -2 the second to last, and so on).
        # This way, we correctly handle the cases where the input tensor and the broadcasted shape
        # differ both in rank and in size along some dimensions.
        expanded_singleton_dims = [i + rank_diff for i, size in enumerate(self.input_shape[::-1]) if size == 1]

        # Combine all indices of singleton dimensions
        singleton_dims = new_singleton_dims + expanded_singleton_dims[::-1]
        return (reshape(summation(out_grad,axes=tuple(singleton_dims)), self.input_shape),) # a deliberate tuple

def broadcast_to(a, shape):
    return BroadcastTo(shape)(a)

In [46]:
import torch

# Create a tensor and set requires_grad to True so that PyTorch will
# know to compute gradients with respect to this tensor.
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# Compute a sum.
y = x.sum()

# Compute gradients.
y.backward()

# The gradient of the sum with respect to x is a tensor of ones.
print(x.grad)  # Outputs: tensor([1., 1., 1.])


tensor([1., 1., 1.])


In [59]:
import torch

# Create a 2D tensor
x = torch.tensor([[1., 2., 3.], [4., 5., 6.]], requires_grad=True)
print("x:\n", x)

# Sum the tensor along axis 0
y = x.sum(axis=0)
print("y:\n", y)
print("y.shape:\n", y.shape)

# Let's define a scalar loss as the sum of all elements in y
loss = y.sum()
print("loss:\n", loss)

# Backward pass
loss.backward()

# Display gradients
print("Gradients:\n", x.grad)

x:
 tensor([[1., 2., 3.],
        [4., 5., 6.]], requires_grad=True)
y:
 tensor([5., 7., 9.], grad_fn=<SumBackward1>)
y.shape:
 torch.Size([3])
loss:
 tensor(21., grad_fn=<SumBackward0>)
Gradients:
 tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [61]:
new_shape = list(x.shape)
new_shape

[2, 3]

In [62]:
for axis in (0,): new_shape[axis] = 1
new_shape

[1, 3]

#### illustrate the reshaping and broadcasting of the gradient in the backward pass.

In [64]:
import torch

# Create a 2D tensor
x = torch.tensor([[1., 2., 3.], [4., 5., 6.]], requires_grad=True)
x

tensor([[1., 2., 3.],
        [4., 5., 6.]], requires_grad=True)

In [65]:
# Sum the tensor along axis 0
y = x.sum(axis=0)
y

tensor([5., 7., 9.], grad_fn=<SumBackward1>)

In [71]:
# Compute a dummy gradient for y
grad_y = torch.ones_like(y)
grad_y, grad_y.shape

(tensor([1., 1., 1.]), torch.Size([3]))

In [68]:
# Reshape grad_y to match the dimensionality of x
new_shape = [1 if axis == 0 else size for axis, size in enumerate(x.shape)]
new_shape

[1, 3]

In [70]:
reshaped_grad_y = grad_y.reshape(new_shape)
reshaped_grad_y, reshaped_grad_y.shape

(tensor([[1., 1., 1.]]), torch.Size([1, 3]))

In [73]:
# Broadcast the reshaped grad_y to match the shape of x
broadcasted_grad_y = reshaped_grad_y.expand_as(x)
broadcasted_grad_y, broadcasted_grad_y.shape

(tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 torch.Size([2, 3]))

## Summation

The `Summation` operation, when provided with the `axes` argument, sums over these axes and thereby reduces the rank of the tensor by the number of axes summed over. The backward pass needs to take this into account, as it needs to return a gradient tensor of the same shape as the input.

Here is a simplified implementation of the correct version:

The forward pass (`compute` method) is straightforward - it just computes the sum over the specified axes.

In the backward pass (`gradient` method), the goal is to compute the gradient of the sum operation. Since every element of the input tensor contributes equally to the sum, the derivative of the sum with respect to each element is 1. However, since the sum operation may reduce the dimensionality of the tensor (when `axes` is not `None`), we need to account for this when computing the gradient.

To do this, we first create a new shape, where the dimensions specified by `axes` are replaced by 1. We then reshape `out_grad` to this new shape. This essentially "undoes" the dimensionality reduction performed by the sum operation. Finally, we use `broadcast_to` to make the reshaped gradient tensor the same shape as the input tensor.

This ensures that the gradient tensor is the correct shape, and that the gradient with respect to each element of the input tensor is correctly computed as 1.

In [106]:
class Summation(TensorOp):
    def __init__(self, axes: Optional[tuple] = None):
        self.axes = axes

    def compute(self, a):
        # Forward pass just computes the sum over the specified axes
        return array_api.sum(a, self.axes)

    def gradient(self, out_grad, node):
        # out_grad is the gradient of the output of this operation
        # We need to "undo" the dimensionality reduction performed in the forward pass
        # That's why we create a new shape, replacing the dimensions specified by self.axes with 1

        # Initialize new shape to be the same as the input shape
        new_shape = list(node.inputs[0].shape)

        # If axes were specified, set those dimensions to 1 in the new shape
        if self.axes:
            for axis in self.axes: new_shape[axis] = 1
                
        # Reshape out_grad to the new shape
        reshaped_grad = reshape(out_grad, new_shape)

        # Broadcast the reshaped out_grad to match the input shape
        broadcasted_grad = broadcast_to(reshaped_grad, node.inputs[0].shape)

        # The gradient method needs to return a tuple, even though there's only one input
        return (broadcasted_grad,)

def summation(a, axes=None):
    return Summation(axes)(a)

## Matrix Multiplication

Matrix multiplication, often denoted by "matmul" in some programming languages, refers to the process of multiplying two matrices together. However, in the context of calculus, it's more common to talk about the derivative of a function. 

When dealing with matrices, instead of talking about derivatives, we often discuss the Jacobian, which is a matrix of partial derivatives. If you have a function that takes a matrix as input and produces a scalar output, you could compute a gradient, which would be a matrix of the same shape as the input matrix.

However, in the context of deep learning and backpropagation, you might be asking about the derivative of a matrix multiplication operation with respect to its inputs. This is often needed when you're training a neural network, because you need to compute gradients to update the weights.

Let's denote the matrices as `A` and `B`, where `A` is a matrix of dimension `m x n` and `B` is a matrix of dimension `n x p`, and the result of the multiplication `C = A * B` is a matrix of dimension `m x p`.

If we are to compute the derivative of `C` with respect to `A` (i.e., ∂C/∂A), each element in `A` affects all elements in its corresponding row in `C`. Thus, the derivative of `C` with respect to `A` is a four-dimensional tensor. In practice, it is common to work with the gradients in a reshaped or unrolled form to perform the necessary update steps in backpropagation.

Similarly, if we are to compute the derivative of `C` with respect to `B` (i.e., ∂C/∂B), each element in `B` affects all elements in its corresponding column in `C`. Again, the derivative will be a four-dimensional tensor.

In actual computation, if we have a scalar-valued loss function `L`, we would compute the gradient of `L` with respect to `A` (denoted as ∂L/∂A), which is the same shape as `A`. To compute this, we need to know the gradient of `L` with respect to `C` (denoted as ∂L/∂C), then:

∂L/∂A = (∂L/∂C) * B^T   (where * denotes matrix multiplication and B^T is the transpose of B)

Similarly, to compute the gradient of `L` with respect to `B` (denoted as ∂L/∂B):

∂L/∂B = A^T * (∂L/∂C)

The details of this process can be quite involved and understanding it fully requires a good understanding of linear algebra and calculus. For more in-depth understanding, it would be beneficial to refer to a textbook or detailed resource on the subject, such as "The Matrix Calculus You Need For Deep Learning" by Terence Parr and Jeremy Howard.


The line `axes_to_sum_over = tuple(range(len(out_shape) - len(lhs_shape)))` is calculating which axes (dimensions) of the output gradient tensor (`out_grad`) need to be summed over when computing the gradient with respect to the left-hand side (`lhs`) input tensor.

This is necessary when the rank (number of dimensions) of `out_grad` is larger than the rank of `lhs`. This can happen, for instance, when `lhs` is a matrix (2D tensor) and `out_grad` is a 3D tensor (which can result from batched matrix multiplication).

The `range` function generates a sequence of integers from 0 up to (but not including) `len(out_shape) - len(lhs_shape)`. The `tuple` function then takes this sequence and turns it into a tuple. The result is a tuple of integers representing the axes to sum over.

Here is a concrete example:

Suppose we have a batched matrix multiplication where `lhs` is a matrix of shape `(m, n)`, and `out_grad` is a 3D tensor of shape `(b, m, n)`, where `b` is the batch size. 

In this case, `len(out_shape) - len(lhs_shape)` equals `1`, so `range(len(out_shape) - len(lhs_shape))` generates a sequence of integers from `0` to `1` (not inclusive), which is just `[0]`.

So `axes_to_sum_over` will be `(0,)`, indicating that we need to sum over the first axis (the batch axis) of `out_grad` when computing the gradient with respect to `lhs`.

This summing operation effectively accumulates the individual gradients for each item in the batch into a single gradient for the `lhs` matrix.

In [82]:
import torch

# Suppose we have the following shapes for `lhs` and `out_grad`
m, n, b = 5, 7, 3

# Let's create some tensors with these shapes
lhs = torch.randn(m, n)          # lhs is a 2D tensor (matrix) of shape (m, n)
out_grad = torch.randn(b, m, n)  # out_grad is a 3D tensor of shape (b, m, n)

# Let's say `rhs` is another matrix that was involved in computing out_grad
rhs = torch.randn(n, m)

# Now we want to compute the gradient of the loss with respect to `lhs`
# First, we transpose `rhs` and perform batched matrix multiplication with `out_grad`
# grad_product = torch.matmul(out_grad, rhs.t())

# # Now we need to sum over the batch dimension
# axes_to_sum_over = (0,)  # in PyTorch, we can also just use 0 instead of a tuple
# grad_wrt_lhs = grad_product.sum(dim=axes_to_sum_over)

# # grad_wrt_lhs is now a tensor of shape (m, n), same as `lhs`
# print(grad_wrt_lhs.shape)  # prints: torch.Size([5, 7])


In [94]:
out_shape, lhs_shape, rsh_shape = out_grad.shape, lhs.shape, rhs.shape
out_shape, lhs_shape, rsh_shape

(torch.Size([3, 5, 7]), torch.Size([5, 7]), torch.Size([7, 5]))

In [101]:
len(out_shape), len(lhs_shape)

(3, 2)

In [103]:
rng = range(len(out_shape) - len(lhs_shape))
rng

range(0, 1)

In [104]:
tuple(rng)

(0,)

In [ ]:
axes_to_sum_over = tuple(range(len(out_shape) - len(lhs_shape)))
axes_to_sum_over

In [107]:
class MatMul(TensorOp):
    def compute(self, a, b):
        return array_api.matmul(a, b)

    def gradient(self, out_grad, node):
        lhs, rhs = node.inputs
        out_shape, lhs_shape, rhs_shape = out_grad.shape, lhs.shape, rhs.shape
        
        # compute gradient with respect to lhs
        if len(lhs_shape) == len(out_shape):
            grad_wrt_lhs = matmul(out_grad, transpose(rhs))
        else:
            axes_to_sum_over = tuple(range(len(out_shape) - len(lhs_shape)))
            grad_wrt_lhs = summation(matmul(out_grad, transpose(rhs)), axes=axes_to_sum_over)
        
        # compute gradient with respect to rhs
        if len(rhs_shape) == len(out_shape):
            grad_wrt_rhs = matmul(transpose(lhs), out_grad)
        else:
            axes_to_sum_over = tuple(range(len(out_shape) - len(rhs_shape)))
            grad_wrt_rhs = summation(matmul(transpose(lhs), out_grad), axes=axes_to_sum_over)
        
        return grad_wrt_lhs, grad_wrt_rhs

def matmul(a, b):
    return MatMul()(a, b)

## Negation

Certainly! Here's the proof and explanation for the derivative of the `Negate` operator:

Let's denote `a` as the tensor being negated. The operation can be described as `f(a) = -a`.

The function for the backward pass (i.e., the gradient) is `df/da = -1`.

The LaTeX document will look as follows:


We are given a function $f(a) = -a$, where $a$ is a tensor. Our task is to find the derivative of this function with respect to $a$.

By differentiating the function $f(a)$ with respect to $a$, we find:

\begin{align*}
\frac{df}{da} &= \frac{d}{da} (-a) \\
&= -1
\end{align*}

Therefore, the gradient of $f(a)$ with respect to $a$ is $-1$.

WE starts by defining the function `f(a) = -a`, where `-` represents the negation operation. It then explains that when we differentiate `f(a)` with respect to `a`, we find that the derivative is `-1`. This means that the gradient of `f(a)` with respect to `a` is `-1`, which matches the behavior of the `Negate` operator.

In [108]:
class Negate(TensorOp):
    def compute(self, a):
        ### BEGIN YOUR SOLUTION
        return -1 * a
        ### END YOUR SOLUTION

    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        return negate(out_grad), 
        ### END YOUR SOLUTION

def negate(a):
    return Negate()(a)

## Log

Certainly! Here's the proof and explanation for the derivative of the `Log` operator:

Let's denote `a` as the tensor on which the logarithm is applied. The operation can be described as `f(a) = \log(a)`, where `\log` represents the natural logarithm.

The function for the backward pass (i.e., the gradient) is `df/da = 1/a`.

The LaTeX document will look as follows:


We are given a function $f(a) = \log(a)$, where $a$ is a tensor. Our task is to find the derivative of this function with respect to $a$.

By differentiating the function $f(a)$ with respect to $a$, we find:

\begin{align*}
\frac{df}{da} &= \frac{d}{da} (\log(a)) \\
&= \frac{1}{a}
\end{align*}

Therefore, the gradient of $f(a)$ with respect to $a$ is $\frac{1}{a}$.


This document starts by defining the function `f(a) = \log(a)`, where `\log` represents the natural logarithm. It then explains that when we differentiate `f(a)` with respect to `a`, we find that the derivative is `1/a`. This means that the gradient of `f(a)` with respect to `a` is `1/a`, which represents the behavior of the `Log` operator.

In [28]:
class Log(TensorOp):
    def compute(self, a):
        ### BEGIN YOUR SOLUTION
        raise array_api.log(a)
        ### END YOUR SOLUTION

    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        return (out_grad / node.inputs[0], )
        ### END YOUR SOLUTION


def log(a):
    return Log()(a)

## Exp

Certainly! Here's the proof and explanation for the derivative of the `Exp` operator:

Let's denote `a` as the tensor on which the exponential function is applied. The operation can be described as `f(a) = \exp(a)`, where `\exp` represents the exponential function.

The function for the backward pass (i.e., the gradient) is `df/da = \exp(a)`.

The LaTeX document will look as follows:


We are given a function $f(a) = \exp(a)$, where $a$ is a tensor. Our task is to find the derivative of this function with respect to $a$.

By differentiating the function $f(a)$ with respect to $a$, we find:

\begin{align*}
\frac{df}{da} &= \frac{d}{da} (\exp(a)) \\
&= \exp(a)
\end{align*}

Therefore, the gradient of $f(a)$ with respect to $a$ is $\exp(a)$.


We starts by defining the function `f(a) = \exp(a)`, where `\exp` represents the exponential function. It then explains that when we differentiate `f(a)` with respect to `a`, we find that the derivative is `\exp(a)`. This means that the gradient of `f(a)` with respect to `a` is `\exp(a)`, which represents the behavior of the `Exp` operator.

In [29]:
class Exp(TensorOp):
    def compute(self, a):
        ### BEGIN YOUR SOLUTION
        self.out = array_api.exp(a)
        return self.out
        ### END YOUR SOLUTION

    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        return out_grad * (exp(node.inputs[0])), 
        ### END YOUR SOLUTION


def exp(a):
    return Exp()(a)

Certainly! Here's the proof and explanation for the derivative of the `ReLU` (Rectified Linear Unit) operator:

Let's denote `a` as the tensor on which the ReLU function is applied. The ReLU function is defined as follows: 

\[
f(a) = 
\begin{cases}
a, & \text{if } a \geq 0 \\
0, & \text{if } a < 0
\end{cases}
\]

The function for the backward pass (i.e., the gradient) is `df/da = 1` if `a >= 0`, and `df/da = 0` if `a < 0`.

The LaTeX document will look as follows:


We are given a function $f(a) = \max(0, a)$, where $a$ is a tensor. Our task is to find the derivative of this function with respect to $a$.

By considering the definition of the ReLU function, we can write $f(a)$ as:

$$
f(a) = 
\begin{cases}
a, & \text{if } a \geq 0 \\
0, & \text{if } a < 0
\end{cases}
$$

Now, let's differentiate $f(a)$ with respect to $a$:

$$
\frac{df}{da} = 
\begin{cases}
1, & \text{if } a \geq 0 \\
0, & \text{if } a < 0
\end{cases}
$$

Therefore, the gradient of $f(a)$ with respect to $a$ is $1$ if $a \geq 0$, and $0$ if $a < 0$.

\end{document}
```

This document starts by defining the function `f(a) = \max(0, a)`, which represents the ReLU function. It then explains that when we differentiate `f(a)` with respect to `a`, we find that the derivative is `1` if `a >= 0`, and `0` if `a < 0`. This means that the gradient of `f(a)` with respect to `a` is `1` for positive values of `a` and `0` for negative values of `a`.

Please note that the `gradient` method of the `ReLU` operator is not implemented in the provided code, as indicated by `NotImplementedError()`.

## ReLU

In [30]:
# TODO
class ReLU(TensorOp):
    def compute(self, a):
        ### BEGIN YOUR SOLUTION
        self.out = array_api.clip(a, a_min=0)
        return self.out
        ### END YOUR SOLUTION

    def gradient(self, out_grad, node):
        ### BEGIN YOUR SOLUTION
        return 1 * self.out
        ### END YOUR SOLUTION


def relu(a):
    return ReLU()(a)